In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
USE_CASES = ["andalusia", "italy", "greece", "poland"]

use_case = "andalusia"

BASE_PATH = f"./../data/use_case_{use_case}"

In [5]:
UC_LINK = {
    "andalusia": "AND",
    "italy": "ITA", 
    "greece": "ELL", 
    "poland": "POL"
}

In [33]:
microdata_ = pd.DataFrame(columns=["A_ID_10_H"])

land_transaction_variables = ["A_ID_10_H", "B_UO_10_A", ]#'B_UT_20_A', 'B_US_30_A',]

for y in range(2014, 2021):
    MICRODATA_FILEPATH = os.path.join(BASE_PATH, f"microdata/AND{y}.csv")

    df_ = pd.read_csv(MICRODATA_FILEPATH)[land_transaction_variables].rename(columns={"B_UO_10_A": f"B_UO_10_A_{y}"})

    df_[f"B_UO_10_A_{y}"] = df_[f"B_UO_10_A_{y}"].apply(lambda x: x/100)

    microdata_ = pd.merge(microdata_, df_, on=["A_ID_10_H"], how="outer")



In [ ]:
microdata_

In [ ]:
for y in range(2014, 2020):
    first_year = y
    second_year = y+1
    print(first_year, second_year)

    microdata_[f"Diff-{first_year}-{second_year}"] = microdata_.apply(lambda x: x[f"B_UO_10_A_{second_year}"] - x[f"B_UO_10_A_{first_year}"], axis=1)


In [ ]:
microdata_


In [ ]:
plt.title("Land transactions distribution")

for c in [cc for cc in microdata_.columns if cc.startswith("Diff")]:
    microdata_[c][
        #(microdata_[c]>-10)&
        #(microdata_[c]<15)&
        (microdata_[c]!=0)
        ].dropna(axis=0).hist(bins=50, label=c)


#plt.xlim(-25, 25)

plt.legend(loc="best")
plt.show()


In [64]:
diff_df = pd.concat([microdata_[c].dropna() for c in microdata_.columns if c.startswith("Diff")]).to_frame().rename(columns={0: "Diff"})

In [ ]:
diff_df[diff_df["Diff"]!=0].hist(bins=100)


In [ ]:
diff_df[diff_df["Diff"]>0].hist(bins=100)


In [ ]:
diff_df[diff_df["Diff"]<0].hist(bins=100)


In [ ]:
diff_df[diff_df["Diff"]>0].describe()

In [ ]:
diff_df[diff_df["Diff"]<0].describe()

In [ ]:
for c in [cc for cc in microdata_.columns if cc.startswith("Diff")]:
    
    print(c)
    print(f"    Min: {microdata_[c].min()}")
    print(f"    Mean: {microdata_[c].mean()}")
    print(f"    Max: {microdata_[c].max()}")

    df_ = microdata_[[c]].dropna()

    n_buy   = round(100*(df_[df_[c]>0].shape[0]/df_.shape[0]), 3)
    n_equal = round(100*(df_[df_[c]==0].shape[0]/df_.shape[0]), 3)
    n_sell  = round(100*(df_[df_[c]<0].shape[0]/df_.shape[0]), 3)

    print(f"\n    n farms buy: {n_buy} %")
    print(f"    n farms neutral: {n_buy}%")
    print(f"    n farms sell: {n_sell}%\n")
    